# Приложение Flask для запроса ChatGPT

Flask application for ChatGPT request



Приглашаю в Телеграм общаться по это теме: https://t.me/AiExp01

In [ ]:
!pip install pyngrok
!pip install openai

In [ ]:
from flask import Flask, request, json
from werkzeug.utils import secure_filename
from pyngrok import ngrok
from google.colab import userdata
from openai import OpenAI
import os

# Установка Ngrok_API_KEY
Ngrok_API_KEY=userdata.get('Ngrok_API_KEY')
ngrok.set_auth_token(Ngrok_API_KEY)

# Установка OpenAI API key
OpenAI_API_KEY=userdata.get('OpenAI_API_KEY')
os.environ["OPENAI_API_KEY"] = OpenAI_API_KEY

# LL_MODEL = "gpt-4-0613"
LL_MODEL = "gpt-3.5-turbo"
print(f'LL_MODEL = {LL_MODEL}')
client = OpenAI(
    api_key=OpenAI_API_KEY
)
port_no = 5000
app = Flask(__name__)

public_url =  ngrok.connect(port_no).public_url
print(f"To acces the Gloable link please click {public_url}")

In [ ]:
# Функции для ChatGPT
def get_answer_gpt(system_content, user_content, temp=0.8):
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]
    completion = client.chat.completions.create(
        model=LL_MODEL,
        messages=messages,
        temperature=temp
    )
    return completion  # возвращает ответ

def get_transcript(file_name, language="en"):
  print('get_transcript')
  print(type(language), f'language={language}')
  # https://platform.openai.com/docs/guides/speech-to-text/transcriptions
  client = OpenAI(api_key=OpenAI_API_KEY)

  audio_file= open(file_name, "rb")
  transcript = client.audio.transcriptions.create(
    model="whisper-1",
    language=language,
    file=audio_file,
    response_format="text"
  )
  print (type(transcript),'\n',transcript)
  return transcript

In [ ]:
# Функции для Flask
UPLOAD_FOLDER = '/content/sample_data'
ALLOWED_EXTENSIONS = {'wav'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route("/")
def home():
    return f"Running Flask on Google Colab!"

@app.route('/chat', methods=['POST'])
def chat():
    data = json.loads(request.data)
    print(type(data), f'chat data={data}')
    system_content = data['system_content']
    user_content = data['user_content']
    completion = get_answer_gpt(system_content, user_content)
    answer = completion.choices[0].message.content
    data['answer'] = answer
    print(f'return data={data}')
    return data


@app.route('/transcript', methods=['POST'])
# https://flask.palletsprojects.com/en/2.3.x/patterns/fileuploads/
def transcript():
    print("transcript")

    # check if the post request has the file part
    if 'file' not in request.files:
        flash('No file part')
        return 'No file part'

    file = request.files['file']
    print(type(file))
    args = request.args
    print(type(args), f'args={args}')
    language = request.args.get('language')
    print(type(language), f'language={language}')

    # If the user does not select a file, the browser submits an
    # empty file without a filename.
    if file.filename == '':
        flash('No selected file')
        return 'No file part'
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    my_file_1 = UPLOAD_FOLDER + "/"+ filename
    print(f'my_file_1={my_file_1}')

    transcript= get_transcript(my_file_1, language=language)

    data = {"transcript": transcript}
    print(f'return data={data}')
    return data

In [ ]:
app.run(port=port_no)

# Вопросы и пожелания сюда:
 https://t.me/AiExp01